# MNIST Classification Experiment Notebook

This Jupyter notebook is a copy of the MNIST classification training script, used for experimentation and tweaking the CNN model. It's purpose is for interactive exploration and quick iterations on the model architecture and hyperparameters.

The actual training script can be found at:
`ML-training-examples/mnist_classification/train.py`

Feel free to adjust parameters, modify the model architecture, or add new features here for experimentation purposes.

In [1]:
import torchvision 
from torchvision.transforms import ToTensor

from torch.utils.data import DataLoader

import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

In [2]:
training_data = torchvision.datasets.MNIST(root="data", transform=ToTensor(), train=True, download=True)
testing_data = torchvision.datasets.MNIST(root="data", transform=ToTensor(), train=False, download=True)

In [3]:
"""
Training set: 60000 images
Testing set: 10000 images

Each 28 x 28 pixels
"""
training_data.data.shape, '|', testing_data.data.shape

(torch.Size([60000, 28, 28]), '|', torch.Size([10000, 28, 28]))

In [4]:
train_dataloader = DataLoader(training_data, batch_size=100, shuffle=True, num_workers=1)
test_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True, num_workers=1)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return x

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using: {device}")


model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

Using: cpu


In [7]:
def train(epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) 
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f"Train epoch: {epoch} [{batch_idx * len(data)} / {len(train_dataloader.dataset)} ({100. * batch_idx / len(train_dataloader):.0f}%)]\t{loss.item():.6f}")

In [8]:
def test():
    model.eval()

    test_loss = 0 
    correct = 0 

    with torch.no_grad():
        for data, target in test_dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_dataloader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(test_dataloader.dataset)} ({100 * correct / len(test_dataloader.dataset):.0f}%\n")                                
    

In [9]:
for epoch in range(1, 6):
    train(epoch)
    test()

Train epoch: 1 [0 / 60000 (0%)]	2.314909
Train epoch: 1 [2000 / 60000 (3%)]	2.205794
Train epoch: 1 [4000 / 60000 (7%)]	1.766574
Train epoch: 1 [6000 / 60000 (10%)]	1.183840
Train epoch: 1 [8000 / 60000 (13%)]	0.825441
Train epoch: 1 [10000 / 60000 (17%)]	0.752452
Train epoch: 1 [12000 / 60000 (20%)]	0.801448
Train epoch: 1 [14000 / 60000 (23%)]	0.869403
Train epoch: 1 [16000 / 60000 (27%)]	0.547647
Train epoch: 1 [18000 / 60000 (30%)]	0.698542
Train epoch: 1 [20000 / 60000 (33%)]	0.649187
Train epoch: 1 [22000 / 60000 (37%)]	0.534904
Train epoch: 1 [24000 / 60000 (40%)]	0.541421
Train epoch: 1 [26000 / 60000 (43%)]	0.471856
Train epoch: 1 [28000 / 60000 (47%)]	0.511495
Train epoch: 1 [30000 / 60000 (50%)]	0.569307
Train epoch: 1 [32000 / 60000 (53%)]	0.410216
Train epoch: 1 [34000 / 60000 (57%)]	0.388296
Train epoch: 1 [36000 / 60000 (60%)]	0.412218
Train epoch: 1 [38000 / 60000 (63%)]	0.365727
Train epoch: 1 [40000 / 60000 (67%)]	0.504049
Train epoch: 1 [42000 / 60000 (70%)]	0.452343